In [113]:
import requests 
import json 
from bs4 import BeautifulSoup
import sys

import pandas as pd 
import numpy as np 
import scipy.stats as sci 

import datetime  
import time 
import random 

import warnings
warnings.filterwarnings(action = 'ignore', category = FutureWarning) 

In [101]:
steamLoginSecure = '76561198329918048%7C%7CeyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MEJGQ18yMTgyMTQ5N182MjQwNCIsICJzdWIiOiAiNzY1NjExOTgzMjk5MTgwNDgiLCAiYXVkIjogWyAid2ViIiBdLCAiZXhwIjogMTY2OTc5ODQwNiwgIm5iZiI6IDE2NjEwNzA2MTIsICJpYXQiOiAxNjY5NzEwNjEyLCAianRpIjogIjBDNjRfMjFBRUM5NTBfQUU4MEUiLCAib2F0IjogMTY2NzAxMTc3NywgInJ0X2V4cCI6IDE2ODQ4NzY5NzUsICJwZXIiOiAwLCAiaXBfc3ViamVjdCI6ICIxMjUuMjM1LjIzOS4xNyIsICJpcF9jb25maXJtZXIiOiAiMTI1LjIzNS4yMzkuMTciIH0.4Yo3Ur2HcAw2FplCBgiWiaehPrpqVcelCa0WUxLEQImcDsyPC3Xi-KLenMG7CWjn-oVQb7dJclU8b5I6e3yoAQ'
cookie = {'steamLoginSecure': steamLoginSecure}

In [21]:
URL = 'https://steamcommunity.com/market/search/render/?query=&start=0&count=100&search_descriptions=0&sort_column=price&sort_dir=desc&appid=730&norender=1'
Items_get = requests.get(url = URL, cookies = cookie)
Items = json.loads(Items_get.text)

data = Items['results']
total_Items = Items['total_count']


In [66]:
print(total_Items)

19157


In [23]:
Items_list = []

In [25]:
# get all items information, because steam only allow 100 items/request, and ~20 requests/minute so everytime request return code 400, put to sleep for 70 seconds
pos = 0
while (pos < total_Items) :
    while (True) :
        URL = 'https://steamcommunity.com/market/search/render/?query=&start=' + str(pos) + '&count=100&search_descriptions=0&sort_column=price&sort_dir=desc&appid=730&norender=1'
        Items_get = requests.get(url = URL, cookies = cookie)
        if(Items_get.status_code != 200) :
            time.sleep(70)
            break
        
        Items = json.loads(Items_get.text)
        data = Items['results']
        for item in data :
            Items_list.append(item['asset_description'])
            
        pos += 100

In [265]:
Item_extra_list = ['StatTrak™', 'Souvenir', 'Normal']

Item_type_list = ['Rifle', 'Sniper Rifle', 'Pistol', 'SMG', 'Shotgun', 'Machinegun', 
                  'Knife', 'Gloves', 'Sticker', 'Container', 'Others']

Exterior_list = ['Battle-Scarred', 'Well-Worn', 'Field-Tested', 'Minimal Wear', 'Factory New', 'None']

Rarity_list = ['Base Grade', 'Consumer Grade', 'Industrial Grade', 'Mil-Spec', 'Restricted', 
               'Classified', 'Covert', 'Extraordinary',
                  'High Grade', 'Remarkable', 'Exotic', 'None']

def _get_type(item_info) :                # get type of an item
    for cur_type in Item_type_list :
        if (item_info.find(cur_type) != -1) : return cur_type
    
    return Item_type_list[-1]

def _get_extra(item_info) :              # get extra status of an item
    for cur_extra in Item_extra_list :
        if (item_info.find(cur_extra) != -1) : return cur_extra
    
    return Item_extra_list[-1]

def _get_rarity(item_info) :              # get rarity of an item
    for cur_rarity in Rarity_list :
        if (item_info.find(cur_rarity) != -1) : return cur_rarity
    
    return Rarity_list[-1]

def _get_exterior(item_description) :     # get exterior of an item
    for cur_exterior in Exterior_list :
        if (item_description.find(cur_exterior) != -1) : return cur_exterior
    
    return Exterior_list[-1]



In [266]:
def _format_ASCII_Name(name) :            # format special characters into ASCII letter
    format_name = ''
    for ch in name :
        if (ch == '|')   : ch = '%7C'
        elif (ch == '&') : ch = '%26'
        elif (ch == ':') : ch = '%3A'
        elif (ch == '/') : ch = '%2F'
        elif (ch == '+') : ch = '%2B'
        elif (ch == ' ') : ch = '%20'
        elif (ch == '(') : ch = '%28'
        elif (ch == ')') : ch = '%29'
        
        format_name += ch
            
    return format_name

In [267]:
def _get_price_request(name) :        # get price data from a skin
    while (True) :
        URL             = 'http://steamcommunity.com/market/pricehistory/?country=PT&currency=3&appid=730&market_hash_name=' + format_name
        item_price_data = requests.get(url = URL, cookies = cookie);
        if (item_price_data.status_code != 200) :
            #print(name, item_price_data.status_code)
            continue
            
        return json.loads(item_price_data.text)
        

In [268]:
items_df = pd.DataFrame(data = None, index = None, columns = ['Name', 'Type', 'Exterior', 'Rarity', 'Extra', 'Average volume sale',
                                                              'Average price (VND)', 'Max price (VND)', 'Days to reach max', 
                                                              'Min price (VND)', 'Days to reach min', 'Price change (%)',
                                                                  'Time on market (days)'])

cnt = 0
for cur_item in Items_list :        # Loop through each item in Item_list
    item_name        = cur_item['market_hash_name']           # get item's name
    item_info        = cur_item['type']                       # get item's information
    item_description = cur_item['descriptions'][0]['value']   # get text contain item's exterior
    
    item_exterior = _get_exterior(item_description)     # get exterior
    item_rarity   = _get_rarity(item_info)              # get rarity
    item_extra    = _get_extra(item_info)               # get extra
    item_type     = _get_type(item_info)                # get type
    
    format_name     = _format_ASCII_Name(item_name)     # format name into ASCII format
    item_price_data = _get_price_request(format_name)   # send request to get price data
    
    if (item_price_data) :                              # if we got information about the item then proceed
        item_prices = item_price_data['prices']         
        success = item_price_data['success']
        if (item_prices == '') | (not item_prices) | (success == False): continue
        
        price_list  = []            # store price each day
        vol_list    = []            # store volume sales each day
        dates       = []            # store dates
        
        tmp_price_list = []         # temp price list
        tmp_vol_list   = []         # temp vol list
        
        last_date = datetime.date(1, 1, 1)
        
        l, r = 0, 0
        
        # There could be data of multiple sales of the same day
        # So we calculate price for a day is the average of all price sale in that day, using tmp_price_list
        # same for vol_list
        
#         print(cnt, item_name)
#         cnt += 1
        
        while (r <= len(item_prices)) :
            cur_date = datetime.date(1, 1, 1)
            if (r < len(item_prices)) :            # still in list's boundary
                cur_date = datetime.datetime.strptime(item_prices[r][0][0 : 11], '%b %d %Y').date()
                
            if (cur_date != last_date and r != 0) :
                avg_price = np.mean(tmp_price_list[l : r])    # get average price of a day
                avg_vol   = np.mean(tmp_vol_list[l : r])      # get average volume sales of a day
                
                price_list.append(avg_price)
                vol_list.append(avg_vol)
                dates.append(last_date)
                
                l = r
            
            if (r < len(item_prices)) : 
                tmp_price_list.append(item_prices[r][1])
                tmp_vol_list.append(float(item_prices[r][2]))
            
            
            last_date = cur_date
            r += 1
        
        avg_price = np.mean(price_list)           # get average price of the item
        avg_vol   = np.mean(vol_list)             # get average sale volumes of the item
        
        max_price = max(price_list)           # get max price of the item 
        min_price = min(price_list)           # get min price of the item
        
        max_idx     = price_list.index(max_price)
        days_to_max = (dates[max_idx] - dates[0]).days   # calculate days to reach maximum
        
        min_idx     = price_list.index(min_price)
        days_to_min = (dates[min_idx] - dates[0]).days   # calculate days to reach minimum
        
        if (price_list[0] == 0) : price_list[0] = 1
        price_change = (price_list[-1] - price_list[0]) / price_list[0]       # calculate percentage of price change from start till now
        price_change = round(price_change, 2)
                                  
        time_on_market   = (dates[-1] - dates[0]).days           # calculate the total days the item has existed
        
        item_dict = {'Name' : item_name, 'Type' : item_type, 'Exterior' : item_exterior, 'Rarity' : item_rarity,
                      'Extra' : item_extra, 'Average volume sale' : avg_vol, 'Average price (VND)' : avg_price,
                        'Max price (VND)' : max_price, 'Days to reach max' : days_to_max,  'Min price (VND)' : min_price, 
                         'Days to reach min' : days_to_min, 'Price change (%)' : price_change, 'Time on market (days)' : time_on_market}
        
        cur_item_df = pd.DataFrame(item_dict, index = [0]);
        items_df    = items_df.append(cur_item_df, ignore_index = True)  
  

#items_df    

0 Sticker | kennyS (Gold) | Boston 2018
1 Souvenir AWP | Safari Mesh (Well-Worn)
2 Glock-18 | Synth Leaf (Well-Worn)
3 ★ Karambit | Lore (Minimal Wear)
4 ★ Specialist Gloves | Emerald Web (Minimal Wear)
5 Negev | Mjölnir (Well-Worn)
6 Souvenir M4A1-S | Knight (Minimal Wear)
7 ★ StatTrak™ Huntsman Knife | Autotronic (Factory New)
8 ★ StatTrak™ Talon Knife | Boreal Forest (Factory New)
9 Sticker | tabseN (Gold) | Krakow 2017
10 ★ Karambit | Boreal Forest (Well-Worn)
11 Sticker | JW (Gold) | Krakow 2017
12 Sticker | dennis (Gold) | Krakow 2017
13 Sticker | chopper (Gold) | Krakow 2017
14 Sticker | nex (Gold) | Krakow 2017
15 Sticker | karrigan (Gold) | Krakow 2017
16 Sticker | hutji (Gold) | Krakow 2017
17 ★ StatTrak™ Skeleton Knife | Fade (Factory New)
18 Sticker | HS (Gold) | Krakow 2017
19 ★ StatTrak™ Stiletto Knife | Damascus Steel (Well-Worn)
20 M4A1-S | Imminent Danger (Factory New)
21 ★ StatTrak™ Karambit | Crimson Web (Minimal Wear)
22 Sticker | markeloff (Gold) | Krakow 2017
23 A

179 Sticker | boltz (Gold) | Krakow 2017
180 Sticker | shox (Gold) | Boston 2018
181 ★ M9 Bayonet | Black Laminate (Battle-Scarred)
182 ★ Butterfly Knife | Crimson Web (Well-Worn)
183 AK-47 | Fire Serpent (Minimal Wear)
184 ★ M9 Bayonet | Gamma Doppler (Minimal Wear)
185 ★ M9 Bayonet | Marble Fade (Factory New)
186 Sticker | Summer (Gold) | Boston 2018
187 Sticker | LUCAS1 (Gold) | Boston 2018
188 ★ StatTrak™ M9 Bayonet | Marble Fade (Factory New)
189 ★ Hand Wraps | Cobalt Skulls (Minimal Wear)
190 ★ StatTrak™ Butterfly Knife | Case Hardened (Minimal Wear)
191 ★ Karambit | Slaughter (Factory New)
192 ★ StatTrak™ M9 Bayonet | Scorched (Battle-Scarred)
193 ★ Karambit | Gamma Doppler (Factory New)
194 ★ StatTrak™ Butterfly Knife | Blue Steel (Minimal Wear)
195 ★ Hand Wraps | Leather (Factory New)
196 ★ StatTrak™ Butterfly Knife | Damascus Steel (Minimal Wear)
197 ★ Butterfly Knife | Lore (Battle-Scarred)
198 ★ M9 Bayonet | Lore (Field-Tested)
199 ★ Karambit | Night (Minimal Wear)
200 AK-4

352 ★ StatTrak™ Nomad Knife | Crimson Web (Minimal Wear)
353 ★ Sport Gloves | Slingshot (Field-Tested)
354 AWP | Fade (Factory New)
355 ★ StatTrak™ Flip Knife | Fade (Factory New)
356 ★ StatTrak™ Karambit | Scorched (Battle-Scarred)
357 ★ StatTrak™ Butterfly Knife | Stained (Minimal Wear)
358 ★ StatTrak™ Karambit | Crimson Web (Well-Worn)
359 ★ M9 Bayonet | Slaughter (Minimal Wear)
360 ★ StatTrak™ Skeleton Knife | Case Hardened (Well-Worn)
361 Souvenir M4A1-S | Imminent Danger (Field-Tested)
362 M4A1-S | Imminent Danger (Well-Worn)
363 ★ Butterfly Knife | Bright Water (Battle-Scarred)
364 ★ StatTrak™ Bowie Knife | Case Hardened (Factory New)
365 ★ StatTrak™ Butterfly Knife | Boreal Forest (Minimal Wear)
366 ★ Bloodhound Gloves | Guerrilla (Factory New)
367 ★ StatTrak™ Skeleton Knife | Crimson Web (Battle-Scarred)
368 Sticker | kennyS (Gold) | London 2018
369 StatTrak™ AWP | Lightning Strike (Factory New)
370 ★ StatTrak™ Butterfly Knife | Case Hardened (Battle-Scarred)
371 ★ StatTrak™ B

520 ★ StatTrak™ Talon Knife | Case Hardened (Minimal Wear)
521 ★ Talon Knife | Tiger Tooth (Factory New)
522 ★ StatTrak™ Huntsman Knife | Lore (Factory New)
523 ★ StatTrak™ Butterfly Knife | Safari Mesh (Field-Tested)
524 Sticker | chrisJ (Gold) | Krakow 2017
525 M4A1-S | Hot Rod (Minimal Wear)
526 ★ M9 Bayonet | Black Laminate (Field-Tested)
527 ★ Sport Gloves | Hedge Maze (Battle-Scarred)
528 ★ Hand Wraps | CAUTION! (Minimal Wear)
529 ★ M9 Bayonet | Damascus Steel (Factory New)
530 ★ M9 Bayonet | Freehand (Minimal Wear)
531 ★ Flip Knife | Crimson Web (Minimal Wear)
532 ★ StatTrak™ Huntsman Knife | Gamma Doppler (Minimal Wear)
533 ★ StatTrak™ Stiletto Knife | Case Hardened (Factory New)
534 ★ M9 Bayonet | Lore (Battle-Scarred)
535 ★ Butterfly Knife | Scorched (Minimal Wear)
536 ★ Butterfly Knife | Boreal Forest (Well-Worn)
537 ★ Bayonet | Marble Fade (Factory New)
538 ★ StatTrak™ Stiletto Knife | Tiger Tooth (Minimal Wear)
539 ★ Driver Gloves | Imperial Plaid (Minimal Wear)
540 ★ Kara

694 Sticker | denis (Gold) | Boston 2018
695 ★ Sport Gloves | Arid (Well-Worn)
696 Sticker | Hobbit (Gold) | Krakow 2017
697 ★ StatTrak™ Skeleton Knife | Case Hardened (Battle-Scarred)
698 ★ StatTrak™ Bayonet | Tiger Tooth (Factory New)
699 ★ Paracord Knife | Crimson Web (Minimal Wear)
700 Sticker | JW (Gold) | Boston 2018
701 ★ StatTrak™ Skeleton Knife | Blue Steel (Well-Worn)
702 ★ Hand Wraps | Arboreal (Factory New)
703 SSG 08 | Sea Calico (Well-Worn)
704 ★ Skeleton Knife | Case Hardened (Well-Worn)
705 Sticker | DickStacy (Gold) | Katowice 2019
706 ★ StatTrak™ Huntsman Knife | Black Laminate (Battle-Scarred)
707 MAG-7 | Cinquedea (Well-Worn)
708 ★ StatTrak™ Stiletto Knife | Marble Fade (Factory New)
709 ★ StatTrak™ M9 Bayonet | Blue Steel (Field-Tested)
710 ★ Karambit | Night (Field-Tested)
711 Sticker | Natus Vincere (Holo) | Katowice 2015
712 Sticker | KrizzeN (Gold) | Boston 2018
713 ★ Driver Gloves | King Snake (Field-Tested)
714 Sticker | LEGIJA (Gold) | Boston 2018
715 Glock-

...

18502 Sealed Graffiti | Noscope (Tiger Orange)
18503 Sealed Graffiti | Mr. Teeth (Frog Green)
18504 Sealed Graffiti | Eye Spy (Brick Red)
18505 Sealed Graffiti | Eco (SWAT Blue)
18506 Sealed Graffiti | Still Happy (Tiger Orange)
18507 Sealed Graffiti | Tilt (Tiger Orange)
18508 Sealed Graffiti | Death Sentence (War Pig Pink)
18509 Sealed Graffiti | Still Happy (Desert Amber)
18510 Sealed Graffiti | Eat It (Monster Purple)
18511 Sealed Graffiti | Worry (Shark White)
18512 Sealed Graffiti | Quickdraw (Shark White)
18513 Sealed Graffiti | Sorry (Battle Green)
18514 Sealed Graffiti | Take Flight (Princess Pink)
18515 Sticker | PGL | Stockholm 2021
18516 Sealed Graffiti | Mr. Teeth (Battle Green)
18517 Sealed Graffiti | X-Axes (Bazooka Pink)
18518 Sealed Graffiti | Noscope (Tracer Yellow)
18519 Sealed Graffiti | Eye Spy (SWAT Blue)
18520 Sealed Graffiti | QQ (Princess Pink)
18521 Sealed Graffiti | Backstab (Frog Green)
18522 Sealed Graffiti | Karambit (Monarch Blue)
18523 Sealed Graffiti | 

18680 Sealed Graffiti | 8-Ball (Princess Pink)
18681 Sealed Graffiti | Worry (Tiger Orange)
18682 Sealed Graffiti | Still Happy (Battle Green)
18683 Sealed Graffiti | GGWP (Desert Amber)
18684 Sealed Graffiti | Sheriff (Tiger Orange)
18685 Sealed Graffiti | X-Axes (Blood Red)
18686 Sealed Graffiti | Toasted (Desert Amber)
18687 Sealed Graffiti | Eye Spy (Tiger Orange)
18688 Sealed Graffiti | Rage Mode (Tracer Yellow)
18689 Sealed Graffiti | X-Axes (Battle Green)
18690 Sealed Graffiti | X-Axes (Violent Violet)
18691 Sealed Graffiti | Ninja (Tracer Yellow)
18692 Sealed Graffiti | Lambda (Blood Red)
18693 Sealed Graffiti | QQ (Tiger Orange)
18694 Sealed Graffiti | Death Sentence (Monarch Blue)
18695 Sealed Graffiti | Double (Violent Violet)
18696 Sealed Graffiti | X-Axes (Shark White)
18697 Sealed Graffiti | Rage Mode (SWAT Blue)
18698 Sealed Graffiti | GTG (Monster Purple)
18699 Sealed Graffiti | GGWP (Violent Violet)
18700 Sealed Graffiti | Death Sentence (Tracer Yellow)
18701 Sealed Gr

18859 Sealed Graffiti | Bling (Cash Green)
18860 Sealed Graffiti | Popdog (Shark White)
18861 Sealed Graffiti | Rage Mode (Battle Green)
18862 Sealed Graffiti | Worry (Tracer Yellow)
18863 Sealed Graffiti | Tilt (Desert Amber)
18864 Sealed Graffiti | Ninja (Cash Green)
18865 Sealed Graffiti | Noscope (Monarch Blue)
18866 Sealed Graffiti | Chess King (Shark White)
18867 Sealed Graffiti | Lambda (Wire Blue)
18868 Sealed Graffiti | Rage Mode (Jungle Green)
18869 Sealed Graffiti | Popdog (Frog Green)
18870 Sealed Graffiti | Jump Shot (Battle Green)
18871 Sealed Graffiti | Bling (Monarch Blue)
18872 Sealed Graffiti | Jump Shot (Violent Violet)
18873 Sealed Graffiti | Eye Spy (Frog Green)
18874 Sealed Graffiti | Chess King (Jungle Green)
18875 Sealed Graffiti | X-Axes (Tracer Yellow)
18876 Sealed Graffiti | Eye Spy (Jungle Green)
18877 Sealed Graffiti | Backstab (Tiger Orange)
18878 Sealed Graffiti | Eco (Violent Violet)
18879 Sealed Graffiti | X-Knives (Tiger Orange)
18880 Sealed Graffiti |

19047 Sticker | drop | Antwerp 2022
19048 Sticker | interz | Stockholm 2021
19049 Sticker | rain | Antwerp 2022
19050 Sticker | Plopski | Stockholm 2021
19051 Sticker | Ninjas in Pyjamas | 2020 RMR
19052 Sticker | faveN | Antwerp 2022
19053 Sticker | buster | Antwerp 2022
19054 Sticker | chopper | Antwerp 2022
19055 Sticker | OG | 2020 RMR
19056 Sticker | hampus | Stockholm 2021
19057 Sticker | buster | Stockholm 2021
19058 Sticker | JOTA | Antwerp 2022
19059 Sticker | Evil Geniuses | Stockholm 2021
19060 Sticker | shox | Stockholm 2021
19061 Sticker | Copenhagen Flames | Antwerp 2022
19062 Sticker | Vitality | Antwerp 2022
19063 Sticker | dgt | Antwerp 2022
19064 Sticker | Techno4K | Antwerp 2022
19065 Sticker | WOOD7 | Antwerp 2022
19066 Sticker | nicoodoz | Antwerp 2022
19067 Sticker | Kyojin | Stockholm 2021
19068 Sticker | stavn | Stockholm 2021
19069 Sticker | refrezh | Stockholm 2021
19070 Sticker | PGL | Antwerp 2022
19071 Sticker | Plopski | Antwerp 2022
19072 Sticker | Qikert

,Name,Type,Exterior,Rarity,Extra,Average volume sale,Average price (VND),Max price (VND),Days to reach max,Min price (VND),Days to reach min,Price change (%),Time on market (days)
0,Sticker | kennyS (Gold) | Boston 2018,Sticker,None,Extraordinary,Normal,2.169355,3.385236e+06,4.254476e+07,1510,6.290441e+05,269,8.62,1582
1,Souvenir AWP | Safari Mesh (Well-Worn),Rifle,Well-Worn,Industrial Grade,Souvenir,1.204947,6.137756e+05,7.803620e+06,2261,1.491100e+01,517,7.40,3238
2,Glock-18 | Synth Leaf (Well-Worn),Pistol,Well-Worn,Restricted,Normal,1.940594,6.527500e+05,4.243951e+06,565,4.135707e+04,0,88.58,1094
3,★ Karambit | Lore (Minimal Wear),Knife,Minimal Wear,Covert,Normal,1.299663,2.178945e+07,4.516186e+07,1806,2.104389e+06,1019,2.55,2326
4,★ Specialist Gloves | Emerald Web (Minimal Wear),Gloves,Minimal Wear,Extraordinary,Normal,1.130435,2.193817e+07,4.602925e+07,2192,7.007052e+06,453,3.71,2192
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19165,Sticker | electroNic | Stockholm 2021,Sticker,None,High Grade,Normal,4362.341970,1.901362e+02,9.322680e+02,0,8.996800e+01,301,-0.86,385
19166,Sticker | misutaaa | Stockholm 2021,Sticker,None,High Grade,Normal,381.709285,1.888014e+02,1.023776e+03,7,8.440136e+01,356,-0.77,385
19167,Sticker | sjuush | Stockholm 2021,Sticker,None,High Grade,Normal,377.094442,1.913917e+02,5.629130e+02,7,8.494800e+01,249,-0.42,385
19168,Sticker | Boombl4 | Stockholm 2021,Sticker,None,High Grade,Normal,5340.321764,2.041169e+02,7.969520e+02,200,9.659396e+01,379,-0.77,385


In [273]:
items_df.to_csv('CSGO_Market.csv', index = False)

In [274]:
tmp_df = pd.read_csv('CSGO_Market.csv')
tmp_df